In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost as xgb
import spacy
from sklearn.svm  import SVC
import mlflow.lightgbm
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.metrics import classification_report,accuracy_score
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
! pip install mlflow boto3 awscli

In [3]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-226-249.eu-north-1.compute.amazonaws.com:5000/")

In [4]:
mlflow.set_experiment("Exp 7 - Best Model-Feature_Eng")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/14', creation_time=1763558084789, experiment_id='14', last_update_time=1763558084789, lifecycle_stage='active', name='Exp 7 - Best Model-Feature_Eng', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [5]:
##READ AND SPLIT##

df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
# Separate features and target
X_cleaned = df['clean_comment']
y_cleaned = df['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned,X_test_cleaned,y_train_cleaned,y_test_cleaned = train_test_split(
    X_cleaned,y_cleaned,test_size=0.2,random_state=42)

In [6]:
# Apply BOW with trigram setting and max_features=5000

BOW = CountVectorizer(ngram_range=(1,3),max_features=5000)
X_train_BOW = BOW.fit_transform(X_train_cleaned).astype('float32')
X_test_BOW = BOW.transform(X_test_cleaned).astype('float32')

In [7]:
! pip install optuna

In [8]:
# function to optimize LightGbm hyperparameter
def objective(trial):
    # Define hyperparameters to be tuned
    param = {
        "objective": "multiclass",
        "num_class":3,
        "learning_rate": trial.suggest_float("learning_rate",1e-3, 1e-1),
        "n_estimators" : trial.suggest_int("n_estimators",50,500),
        "min_child_samples": trial.suggest_int('min_child_samples', 10, 200),
        "reg_lambda" : trial.suggest_float('reg_lambda', 1e-4, 50.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
    }

    # CHANGES MADE: Start nested MLflow run for this trial
    with mlflow.start_run(nested=True):
        # Log trial hyperparameters
        for k, v in param.items():
            mlflow.log_param(k, v)
     
        #Define the Lightgbm parameter with the trial parameter
        model = lgb.LGBMClassifier(**param)

        #perform a cross validation
        scores =cross_val_score(model,X_train_BOW,y_train_cleaned,cv=3,scoring='accuracy')

        # Return the average score across folds
        mean_score = scores.mean()

        # Log the mean CV accuracy
        mlflow.log_metric("mean_cv_accuracy", mean_score)

    return mean_score

In [ ]:

# ---------------------------
# Start MLflow run
# ---------------------------
with mlflow.start_run(run_name="Stacking_Ensemble_Reddit"):

    # Create an Optuna study to optimize the hyperparameters
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    # Log best trial params
    best_params = study.best_trial.params
    for k, v in best_params.items():
        mlflow.log_param(f"best_{k}", v)
    
    
    # ---------------------------
    # Define base learners
    # ---------------------------
    lightgbm_model = LGBMClassifier(
        objective='multiclass',
        num_class=3,
        metric="multi_logloss",
        is_unbalance=True,
        class_weight="balanced",
        random_state=42,
        **best_params
    )

    xgboost_model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        learning_rate=0.08,
        n_estimators=400,
        max_depth=6,
        eval_metric='mlogloss',
        use_label_encoder=False,
        verbosity=0,
        random_state=42
    )

    svm_model = make_pipeline(
        StandardScaler(with_mean=False),
        SVC(kernel='rbf',
        C=1.0,
        probability=True,  # Needed for stacking
        class_weight='balanced',
        random_state=42)
    )

    # ---------------------------
    # Define meta-learner
    # ---------------------------
    meta_learner = LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        solver='lbfgs',
        multi_class='multinomial',
        random_state=42
    )

    # ---------------------------
    # Define stacking classifier
    # ---------------------------
    stacking_model = StackingClassifier(
        estimators=[
            ('lightgbm', lightgbm_model),
            ('xgboost', xgboost_model),
            ('svm', svm_model)
        ],
        final_estimator=meta_learner,
        cv=3,
        n_jobs=-1,
        passthrough=True
    )

    # ---------------------------
    # Fit the stacking model
    # ---------------------------
    stacking_model.fit(X_train_BOW, y_train_cleaned)

    # ---------------------------
    # Predict and evaluate
    # ---------------------------
    y_pred = stacking_model.predict(X_test_BOW)
    accuracy = accuracy_score(y_test_cleaned, y_pred)
    report = classification_report(y_test_cleaned, y_pred, output_dict=True)

    # ---------------------------
    # Log metrics to MLflow
    # ---------------------------
    mlflow.log_metric("test_accuracy", accuracy)
    for cls in ["0", "1", "2"]:
        mlflow.log_metric(f"precision_class_{cls}", report[cls]["precision"])
        mlflow.log_metric(f"recall_class_{cls}", report[cls]["recall"])
        mlflow.log_metric(f"f1_class_{cls}", report[cls]["f1-score"])

    # ---------------------------
    # Log base learner parameters
    # ---------------------------
    mlflow.log_param("lightgbm_learning_rate", lightgbm_model.learning_rate)
    mlflow.log_param("lightgbm_n_estimators", lightgbm_model.n_estimators)
    mlflow.log_param("lightgbm_max_depth", lightgbm_model.max_depth)

    mlflow.log_param("xgboost_learning_rate", xgboost_model.learning_rate)
    mlflow.log_param("xgboost_n_estimators", xgboost_model.n_estimators)
    mlflow.log_param("xgboost_max_depth", xgboost_model.max_depth)

    mlflow.log_param("svm_kernel", svm_model.kernel)
    mlflow.log_param("svm_C", svm_model.C)

    # ---------------------------
    # Log the final stacking model
    # ---------------------------
    mlflow.sklearn.log_model(stacking_model, artifact_path="stacking_model")

    print("Stacking Model Test Accuracy:", accuracy)
    print(report)


[I 2025-11-19 22:19:04,083] A new study created in memory with name: no-name-fb354065-e94f-421d-bfc4-9808745dbe33


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011901 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2091
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 390
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2089
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 382
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3626
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 748
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🏃 View run zealous-gull-155 at: http://ec2-13-62-226-249.eu-north-1.compute.amazonaws.com:5000/#/experiments/14/runs/1d8c74fbfe8149d499e193e8f3b5598b
🧪 View experiment at: http://ec2-13-62-226-249.eu-north-1.compute.amazonaws.com:5000/#/experiments/14


[I 2025-11-19 22:52:04,958] Trial 15 finished with value: 0.7636127779878636 and parameters: {'learning_rate': 0.07377950030409679, 'n_estimators': 230, 'min_child_samples': 80, 'reg_lambda': 0.01712750950153498, 'max_depth': 20}. Best is trial 13 with value: 0.8208256999333919.
